In [1]:
import hyperparams
import mlp_regressor
from mlp_regressor import MLPRegressor

# Load vectors

In [2]:
import json

import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset, DataLoader

In [3]:
# Load X data
INPUT_NPZ_PATH = 'data/filtered_vectors.npz' 

try:
    with np.load(INPUT_NPZ_PATH) as data:
        norm_reps = data['representations_normalized']
        row_indices = data['row_indices']
        
except FileNotFoundError:
    print(f"Error: The file '{INPUT_NPZ_PATH}' was not found.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [4]:
# Load y data

with open('data/score_by_entity.json', encoding='utf-8') as file:
    data = json.load(file)

# Create a dictionary mapping vector_index to score for O(1) lookup
score_dict = {item['vector_index']: item['score'] for item in data}

# Create scores list in the order determined by row_indices
scores = np.array([score_dict[idx] for idx in row_indices])

In [5]:
# Load stratification data
with open('data/score_by_entity.json', encoding='utf-8') as file:
    data = json.load(file)

# Create a dictionary mapping vector_index to score for O(1) lookup
country_dict = {item['vector_index']: item['pais'] for item in data}

# Create scores list in the order determined by row_indices
countries = np.array([country_dict[idx] for idx in row_indices])

In [6]:
X, y = norm_reps, scores

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=hyperparams.RANDOM_SEED, stratify=countries
    )

# Convert to PyTorch tensors and create DataLoader
train_dataset = TensorDataset(
    torch.tensor(X_train, dtype=torch.float32),
    torch.tensor(y_train, dtype=torch.float32)
)

# Create DataLoader with batch size
train_loader = DataLoader(train_dataset, batch_size=hyperparams.BATCH_SIZE, shuffle=True)

# Probe model

In [10]:
import os

In [8]:
# Instantiate MLPRegressor with the hyperparameters
model = MLPRegressor(
    input_size=X.shape[1],
    optimizer=hyperparams.OPTIM,
    learning_rate=hyperparams.LR,
    max_iter=hyperparams.MAX_ITER
)
model.cuda()

MLPRegressor(
  (criterion): MSELoss()
  (layers): ModuleList(
    (0): Linear(in_features=3072, out_features=1, bias=False)
  )
)

In [9]:
model.fit(train_loader, y_train, X_test, y_test)

Epoch 1/100
{'test_loss': 0.029070734977722168, 'train_loss': 0.0009764875212287752, 'test_pearson_corr': np.float32(-0.0491274)}
Epoch 2/100
{'test_loss': 0.029070734977722168, 'train_loss': 0.0009682430142398965, 'test_pearson_corr': np.float32(0.011457597)}
Epoch 3/100
{'test_loss': 0.029070734977722168, 'train_loss': 0.0009545247060340857, 'test_pearson_corr': np.float32(0.012163753)}
Epoch 4/100
{'test_loss': 0.029070734977722168, 'train_loss': 0.0009542764144945352, 'test_pearson_corr': np.float32(0.012845432)}
Epoch 5/100
{'test_loss': 0.029070734977722168, 'train_loss': 0.0009549964561176837, 'test_pearson_corr': np.float32(0.013502194)}
Epoch 6/100
{'test_loss': 0.029070734977722168, 'train_loss': 0.0009548368743918205, 'test_pearson_corr': np.float32(0.014133836)}
Epoch 7/100
{'test_loss': 0.029070734977722168, 'train_loss': 0.0009558282589571185, 'test_pearson_corr': np.float32(0.01474297)}
Epoch 8/100
{'test_loss': 0.029070734977722168, 'train_loss': 0.0009546013247823856, 

c:\Users\sofia\Documents\genai\keen-probe-latinoamerica\mlp_regressor.py:88: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  test_pearson_corr, test_pearson_p_value = pearsonr(result_df["preds"], result_df["target"])


{'test_loss': 0.029070734977722168, 'train_loss': 0.0010553333719869823, 'test_pearson_corr': np.float32(nan)}
Epoch 54/100
{'test_loss': 0.029070734977722168, 'train_loss': 0.0009542764139070683, 'test_pearson_corr': np.float32(nan)}
Epoch 55/100
{'test_loss': 0.029070734977722168, 'train_loss': 0.0009558606210781312, 'test_pearson_corr': np.float32(nan)}
Epoch 56/100
{'test_loss': 0.029070734977722168, 'train_loss': 0.0009550927386142103, 'test_pearson_corr': np.float32(nan)}
Epoch 57/100
{'test_loss': 0.029070734977722168, 'train_loss': 0.0009553109762199621, 'test_pearson_corr': np.float32(nan)}
Epoch 58/100
{'test_loss': 0.029070734977722168, 'train_loss': 0.0009549126706139686, 'test_pearson_corr': np.float32(nan)}
Epoch 59/100
{'test_loss': 0.029070734977722168, 'train_loss': 0.0009544797913107783, 'test_pearson_corr': np.float32(nan)}
Epoch 60/100
{'test_loss': 0.029070734977722168, 'train_loss': 0.00095521428101289, 'test_pearson_corr': np.float32(nan)}
Epoch 61/100
{'test_los

In [11]:
save_dir = 'models'
os.makedirs(save_dir, exist_ok=True)

# recommended: save state_dict (binary)
state_path = os.path.join(save_dir, 'mlp_regressor_state_dict.pth')
torch.save(model.state_dict(), state_path)

print(f"Saved state_dict -> {state_path}")

Saved state_dict -> models\mlp_regressor_state_dict.pth
